In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import libraries

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import pandas as pd

In [ ]:
# insert your path
path='./drive/MyDrive/Colab_Notebooks/Datamining_Project/'

Read the dataset with the overall data about pressure.

In [ ]:
# only to read data
tot = pd.read_csv(path+'tot_pressure.csv',index_col=['Points:0','Points:1','Points:2'])
tot_df = tot.T
tot = tot_df.to_numpy()

Define the function to prepare data for the CNN model.

In [ ]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, Y = list(), list()
    for i in range(len(sequences)+1):
        # find the end of this pattern
        end_ix = i + n_steps
        
        # check if we are beyond the dataset
        if end_ix > len(sequences)-1:
            break
		
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
        X.append(seq_x)
        Y.append(seq_y)
        
    return np.array(X), np.array(Y) # the outputs of the function are, respectively, the input data and the target data of the model

Splitting data into training set - test set

In [ ]:
# Training set - Test set
# 80% - 20%, choosed chronologically

tot_train = tot[:800,:]
tot_test = tot[800:,:]

Build the model

In [ ]:
# choose a number of time steps
n_steps = 75
# convert into input/output
X_train, Y_train = split_sequences(tot_train, n_steps)
n_features = X_train.shape[2]
# define model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='tanh', input_shape=(n_steps, n_features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='tanh'))
model.add(Dense(n_features))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X_train, Y_train, epochs=3000, verbose=0)
# convert into input/output
X_test, Y_test = split_sequences(tot_test, n_steps)

nan


Finally compute the prediction

In [ ]:
Y_pred = model.predict(X_test, verbose=0)

Save data

In [ ]:
# only to save data
# pd.DataFrame(Y_pred).to_csv(path + "Y_pres_cnn.csv", index=False)

AttributeError: ignored

In [ ]:
# only to read data
# Y_pred = pd.read_csv(path+'Y_pres_cnn.csv')

Create 126 csv files with the predictions.

In [ ]:
# to save results
Y_pred = Y_pred.to_numpy()
data_pred=tot_df.T.reset_index()[['Points:0','Points:1','Points:2']]

In [ ]:
# save data: a dataset is created for each second and it contains the prediction for every point
for i in range(len(Y_pred)):
  j=i+n_steps
  if i==0:
    data_pred.insert(0,'Predicted p',Y_pred[i])
    data_pred.to_csv(path+"dataCNN/dataPred_"+str(1000-200+j)+".csv", index=False)
  if i!=0:
    data_pred['Predicted p']=Y_pred[i]
    data_pred.to_csv(path+"dataCNN/dataPred_"+str(1000-200+j)+".csv", index=False)